In [1]:
import os
import absl
import urllib
import tempfile

import tensorflow as tf
import tfx

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

tf.get_logger().propagate = False
absl.logging.set_verbosity(absl.logging.INFO)

In [2]:
_data_root = tempfile.mkdtemp()
_data_filepath = os.path.join(_data_root, 'data.csv')
DATA_PATH = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv'
urllib.request.urlretrieve(DATA_PATH, _data_filepath)

('/tmp/tmp4vcd__pm/data.csv', <http.client.HTTPMessage at 0x7f00fb6d1210>)

In [3]:
context = InteractiveContext()

In [4]:
from tfx.utils.dsl_utils import external_input
from tfx.components import CsvExampleGen
example_gen = CsvExampleGen(input=external_input(_data_root))
context.run(example_gen)

INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.
INFO:absl:Using 1 process(es) for Beam pipeline execution.
INFO:absl:Processing input csv data /tmp/tmp4vcd__pm/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: /tmp/tfx-interactive-2020-05-16T01_54_55.608532-y57pihai/CsvExampleGen/examples/1, id: 2)]
        ))

In [5]:
from tfx.components import StatisticsGen
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
context.run(statistics_gen)

INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Using 1 process(es) for Beam pipeline execution.
INFO:absl:Generating statistics for split train
INFO:absl:Statistics for split train written to /tmp/tfx-interactive-2020-05-16T01_54_55.608532-y57pihai/StatisticsGen/statistics/2/train.
INFO:absl:Generating statistics for split eval
INFO:absl:Statistics for split eval written to /tmp/tfx-interactive-2020-05-16T01_54_55.608532-y57pihai/StatisticsGen/statistics/2/eval.
/opt/conda/lib/python3.7/site-packages/tensorflow_data_validation/arrow/arrow_util.py:239: FutureWarning: Calling .data on ChunkedArray is provided for compatibility after Column was removed, simply drop this attribute
  types.FeaturePath([column_name]), column.data.chunk(0), weights):
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(type_name: ExampleStatistics, uri: /tmp/tfx-interactive-2020-05-16T01_54_55.608532-y57pihai/StatisticsGen/statistics/2, id: 3)]
        ))

In [6]:
from tfx.components import SchemaGen
schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'])
context.run(schema_gen)

INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for SchemaGen
INFO:absl:Infering schema from statistics.


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


INFO:absl:Schema written to /tmp/tfx-interactive-2020-05-16T01_54_55.608532-y57pihai/SchemaGen/schema/3/schema.pbtxt.
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(type_name: Schema, uri: /tmp/tfx-interactive-2020-05-16T01_54_55.608532-y57pihai/SchemaGen/schema/3, id: 4)]
        ))

In [7]:
from tfx.components import Transform

transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath('taxi_transform.py'),
    instance_name='transform'
)
context.run(transform)

INFO:absl:Running driver for Transform.transform
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Transform.transform


Instructions for updating:
Schema is a deprecated, use schema_utils.schema_from_feature_spec to create a `Schema`


INFO:absl:Using 1 process(es) for Beam pipeline execution.


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' object has no attribute 'name'
INFO:tensorflow:SavedModel written to: /tmp/tfx-interactive-2020-05-16T01_54_55.608532-y57pihai/Transform.transform/transform_graph/4/.temp_path/tftransform_tmp/c40d7428d3a44f7b80c649e58c9538ec/saved_model.pb
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' object has no attribute 'name'
INFO:tensorflow:SavedModel written to: /tmp/tfx-interactive-2020-05-16T01_54_55

INFO:absl:Running publisher for Transform.transform
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Transform.transform
    execution_id: 4
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(type_name: TransformGraph, uri: /tmp/tfx-interactive-2020-05-16T01_54_55.608532-y57pihai/Transform.transform/transform_graph/4, id: 5)]
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: /tmp/tfx-interactive-2020-05-16T01_54_55.608532-y57pihai/Transform.transform/transformed_examples/4, id: 6)]
        ))

Trainer takes:

- tf.Examples used for training and eval.
- A user provided module file that defines the trainer logic.
- A data schema created by a SchemaGen pipeline component and optionally altered by the developer.
- Protobuf definition of train args and eval args.
- (Optional) transform graph produced by an upstream Transform component.
- (Optional) pre-trained models used for scenarios such as warmstart.

Trainer emits:

- At least one model for inference/serving (typically in SavedModelFormat) and optionally another model for eval (typically an EvalSavedModel).

## Estimator based Trainer

### Configuring a Trainer Component

In [ ]:
from tfx.components import Trainer
from tfx.proto import trainer_pb2

## parameter specification:
# module_file: A path to python module file containing UDF model definition.

#     For default executor, The module_file must implement a function named
#     `trainer_fn` at its top level. The function must have the following
#     signature.

#     def trainer_fn(trainer.executor.TrainerFnArgs,
#                    tensorflow_metadata.proto.v0.schema_pb2) -> Dict:
#       ...

#     where the returned Dict has the following key-values.
#       'estimator': an instance of tf.estimator.Estimator
#       'train_spec': an instance of tf.estimator.TrainSpec
#       'eval_spec': an instance of tf.estimator.EvalSpec
#       'eval_input_receiver_fn': an instance of
#         tfma.export.EvalInputReceiver.

#     Exactly one of 'module_file' or 'trainer_fn' must be supplied.

#     For generic executor, The module_file must implement a function named
#     `run_fn` at its top level with function signature:
#     `def run_fn(trainer.executor.TrainerFnArgs)`, and the trained model must
#     be saved to TrainerFnArgs.serving_model_dir when execute this function.

trainer = Trainer(
    examples=transform.outputs['transformed_examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath('taxi_trainer.py'),
    train_args=trainer_pb2.TrainArgs(num_steps=10000),
    eval_args=trainer_pb2.EvalArgs(num_steps=5000),
    transform_graph=transform.outputs['transform_graph'], # optional produced by upstream Transform component
#     base_model=..., # optional for warmstart
)
context.run(trainer)

### Generic Trainer

Generic trainer enables developers to use any TensorFlow model API with the Trainer component. In addition to TensorFlow Estimators, developers can use Keras models or custom training loops. For details, please see the [RFC for generic trainer](https://github.com/tensorflow/community/blob/master/rfcs/20200117-tfx-generic-trainer.md).

In [ ]:
from tfx.components import Trainer
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor

trainer = Trainer(
    examples=transform.outputs['transformed_examples'],
    schema=schema_gen.outputs['schema'],
    module_file=..., # `run_fn` in this file
    train_args=trainer_pb2.TrainArgs(num_steps=10000),
    eval_args=trainer_pb2.EvalArgs(num_steps=5000),
    transform_graph=transform.outputs['transform_graph'],
    custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor)
)